In [15]:
import numpy as np
import math
from numpy import linalg as LA

## Treci domaci: Izometrije prostora

#### 1. funkcija: Euler2A
- kao ulazne parametre prima tri Ojlerova ugla i za njih racuna matricu A

In [16]:
def Euler2A(fi, teta, psi):

    Rz = np.array([[math.cos(psi), -math.sin(psi), 0],
                   [math.sin(psi), math.cos(psi), 0],
                   [0, 0, 1]])

    Ry = np.array([[math.cos(teta), 0, math.sin(teta)],
                   [0, 1, 0],
                   [-math.sin(teta), 0, math.cos(teta)]])

    Rx = np.array([[1, 0, 0],
                   [0, math.cos(fi), -math.sin(fi)],
                   [0, math.sin(fi), math.cos(fi)]])

    return Rz.dot(Ry).dot(Rx)


#### 2. funkcija: AxisAngle
- predstavlja matricu A preko ose i ugla

In [17]:
def AxisAngle(A):
    
    if round(LA.det(A)) != 1:
        print("Determinanta je razlicita od 1")
        return

    if np.any(np.round(A.dot(A.T),6) != np.eye(3)):
        print("Matrica A nije ortogonalna")
        return
    
    lambdas, vector = np.linalg.eig(A, )
    for i in range(len(lambdas)):
        if np.round(lambdas[i], 6) == 1.0:
            p = np.real(vector[:, i])

    p1 = p[0]
    p2 = p[1]
    p3 = p[2]
    if p1 == 0 and p2 == 0 and p3 == 0:
        print("Ne smeju sve tri koordinate da budu 0")
        return
    elif p1 != 0:
        u = np.cross(p, np.array([-p1, p2, p3]))
    elif p2 != 0:
        u = np.cross(p, np.array([p1, -p2, p3]))
    else:
        u = np.cross(p, np.array([p1, p2, -p3]))
    u = u/math.sqrt(u[0]**2+u[1]**2+u[2]**2)

    up = A.dot(u)

    fi = np.round(math.acos(np.sum(u*up)), 5)
    if np.round(np.sum(np.cross(u, up)*p), 5) < 0:
        p = (-1)*p

    return [p, fi]

#### 3. funkcija: Rodrigez
- za zadatu osu i ugao odredjuje matricu A

In [18]:
def Rodrigez(p, fi):

    p1 = p[0]
    p2 = p[1]
    p3 = p[2]

    Px = np.array([[0, -p3, p2],
                   [p3, 0, -p1],
                   [-p2, p1, 0]])

    E = np.eye(3)
    p = np.reshape(p, (3, 1))
    Rp = p.dot(p.T) + math.cos(fi)*(E - p.dot(p.T)) + math.sin(fi)*Px

    return Rp

#### 4. funkcija: A2Euler
- za zadatu matricu A odredjuje njene Ojlerove uglove

In [19]:
def A2Euler(A):
    if round(LA.det(A)) != 1:
        print("Determinanta je razlicita od 1")
        return

    if np.any(np.round(A.dot(A.T),6) != np.eye(3)):
        print("Matrica A nije ortogonalna")
        return
    
    fi, teta, psi = 0, 0, 0
    if A[2, 0] < 1:
        if A[2, 0] > -1:
            psi = math.atan2(A[1, 0], A[0, 0])
            teta = math.asin((-1)*A[2, 0])
            fi = math.atan2(A[2, 1], A[2, 2])
        else:
            psi = math.atan2((-1)*A[0, 1], A[1, 1])
            teta = math.pi/2.0
            fi = 0.0
    else:
        psi = math.atan2((-1)*A[0, 1], A[1, 1])
        teta = (-1.0)*math.pi/2.0
        fi = 0

    return([fi, teta, psi])

#### 5. funkcija: AxisAngle2Q
- preko ose i ugla odredjuje jedinicni kvaternion

In [20]:
def AxisAngle2Q(p, fi):
    w = math.cos(fi/2.0)
    norm = np.linalg.norm(p)
    if norm != 0:
        p = p/norm
    [x, y, z] = math.sin(fi/2.0) * p
    return [x, y, z, w]

#### 6. funkcija: Q2AxisAngle
- zadati jedinicni kvaternion vraca u osu i ugao

In [21]:
def Q2AxisAngle(q):
    if q[0] == 0 and q[1] == 0 and q[2] == 0:
        print("Kvaternion ne sme biti 0")
        return
    
    #normalizujemo kvaternion
    norm = np.linalg.norm(q)
    if norm != 0:
        q = q/norm

    fi = 2*math.acos(q[3])
    if abs(q[3]) == 1:
        p = [1, 0, 0]
    else:
        norm = np.linalg.norm(np.array([q[0], q[1], q[2]]))
        p = np.array([q[0], q[1], q[2]])
        if norm != 0:
            p = p / norm

    return [p, fi]

### Testiranje funkcija:

In [22]:
fi = -math.atan(1/4)
print("Ugao fi:",fi)

teta = -math.asin(8/9)
print("Ugao teta:",teta)
psi = math.atan(4)

print("Ugao psi:", psi)
print()

print("Matrica A (dobijena funkcijom Euler2A):\n ")
A = Euler2A(fi, teta, psi)
print(A, "\n")

print("Matrica A predstavljena preko ose i ugla (dobijena funkcijom AxisAngle): \n")
N = AxisAngle(A)
print(N, "\n")

p = N[0] #osa
fi = N[1] #ugao

print("Za dobijenu osu i ugao odredjujemo matricu A (funkcijom Rodrigez): \n")
A_new = Rodrigez(p,fi)
print(A_new, "\n")

print("Odredjujemo Ojlerove uglove za matricu A (funkcijom A2Euler): \n")
X = A2Euler(A)
print(X, "\n")

print("Preko ose i ugla dobijamo jedinicni kvaternion (funkcijom AxisAngle2Q): \n")
q = AxisAngle2Q(p,fi)
print(q, "\n")

print("Dobijeni kvaternion vracamo u osu i ugao (funkcijom Q2AxisAngle): \n")
X = Q2AxisAngle(q)
print(X, "\n")

Ugao fi: -0.24497866312686414
Ugao teta: -1.09491407713448
Ugao psi: 1.3258176636680326

Matrica A (dobijena funkcijom Euler2A):
 
[[ 0.11111111 -0.88888889 -0.44444444]
 [ 0.44444444  0.44444444 -0.77777778]
 [ 0.88888889 -0.11111111  0.44444444]] 

Matrica A predstavljena preko ose i ugla (dobijena funkcijom AxisAngle): 

[array([ 0.33333333, -0.66666667,  0.66666667]), 1.5708] 

Za dobijenu osu i ugao odredjujemo matricu A (funkcijom Rodrigez): 

[[ 0.11110785 -0.88888971 -0.44444363]
 [ 0.44444363  0.4444424  -0.77777941]
 [ 0.88888971 -0.11111274  0.4444424 ]] 

Odredjujemo Ojlerove uglove za matricu A (funkcijom A2Euler): 

[-0.24497866312686414, -1.09491407713448, 1.3258176636680326] 

Preko ose i ugla dobijamo jedinicni kvaternion (funkcijom AxisAngle2Q): 

[0.23570269328649118, -0.4714053865729824, 0.4714053865729824, 0.7071054825112363] 

Dobijeni kvaternion vracamo u osu i ugao (funkcijom Q2AxisAngle): 

[array([ 0.33333333, -0.66666667,  0.66666667]), 1.5708] 

